# ROOP-FLOYD Environment Check

This notebook helps verify your ROOP-FLOYD installation is working correctly on Vast.ai

## 1. Check System Information

In [ ]:
!nvidia-smi

In [ ]:
!python -c "import platform; print('Python version:', platform.python_version())"

In [ ]:
!df -h

## 2. Verify Torch and CUDA

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

## 3. Check ONNX Runtime

In [ ]:
import onnxruntime as ort
print(f"ONNX Runtime version: {ort.__version__}")
print(f"Available providers: {ort.get_available_providers()}")

## 4. Verify Required Libraries

In [ ]:
!pip list | grep -E 'gradio|opencv|insightface|numpy'

## 5. Fix Import Order (If Needed)

In [ ]:
# Create preload.py if it doesn't exist
!if [ ! -f "/roop-floyd/preload.py" ]; then echo -e 'import torch\nimport onnxruntime\nprint("Preloaded torch and onnxruntime in the correct order")' > /roop-floyd/preload.py; fi

# Check if run.py already imports preload
!if ! grep -q "import preload" "/roop-floyd/run.py"; then sed -i '1s/^/import preload\n/' "/roop-floyd/run.py"; fi

print("Import order check complete")

## 6. Check Settings (server_share should be True)

In [ ]:
import os
import yaml

# Function to check settings.py
def check_settings_py():
    try:
        with open('/roop-floyd/settings.py', 'r') as f:
            content = f.read()
            if 'self.server_share = self.default_get(data, \'server_share\', True)' in content:
                print("settings.py: server_share is set to True ✓")
            else:
                print("settings.py: server_share is NOT set to True ⨯")
    except Exception as e:
        print(f"Error reading settings.py: {e}")

# Function to check settings.yaml if it exists
def check_settings_yaml():
    try:
        if os.path.exists('/roop-floyd/settings.yaml'):
            with open('/roop-floyd/settings.yaml', 'r') as f:
                data = yaml.safe_load(f)
                if data and 'server_share' in data and data['server_share'] is True:
                    print("settings.yaml: server_share is set to True ✓")
                else:
                    print("settings.yaml: server_share is NOT set to True ⨯")
        else:
            print("settings.yaml doesn't exist yet, will be created on first run")
    except Exception as e:
        print(f"Error reading settings.yaml: {e}")

check_settings_py()
check_settings_yaml()

## 7. Fix Settings.py (if needed)

In [ ]:
!sed -i "/^\s*self\.server_share\s*=\s*self\.default_get(.*,\s*'server_share',\s*False)/c\        self.server_share = self.default_get(data, 'server_share', True)" "/roop-floyd/settings.py"
print("Updated settings.py")

## 8. Start ROOP-FLOYD Server

In [ ]:
# This will block until the server is stopped
!cd /roop-floyd && python run.py